# Data Cleaning

## Text Extraction

In [1]:
%pip install pandas requests beautifulsoup4 openpyxl

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

df = pd.read_excel('web_scraping_urls.xlsx', sheet_name='2020 - 2024')

def clean_text(text):
    return re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)

def extract_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return clean_text(soup.get_text())
    except requests.RequestException as e:
        return str(e)

batch_size = 10
output_filename = 'all_extracted_texts.xlsx'

with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    total_urls = len(df['URLs'])
    for start in range(180, total_urls, batch_size):
        end = start + batch_size
        print(f"processing batch from URL {start+1} to {min(end, total_urls)}")
        batch_data = {'URL': df['URLs'][start:end], 'Quarter': df['Quarter'][start:end], 'Extracted Text': []}

        for index, url in enumerate(df['URLs'][start:end], start=start+1):
            text = extract_text(url)
            batch_data['Extracted Text'].append(text)
            print(f"extracted from URL {index}: {text[:60]}...")
            time.sleep(2)

        batch_df = pd.DataFrame(batch_data)
        print(batch_df.head())  # Print head of the DataFrame to check it
        batch_df.to_excel(writer, sheet_name='2020 - 2024', index=False, header=not writer.sheets, startrow=writer.sheets['2020 - 2024'].max_row if '2020 - 2024' in writer.sheets else 0)

        print(f"batch from URL {start+1} to {min(end, total_urls)} saved.")


processing batch from URL 181 to 190
extracted from URL 181:    Moomoo SG - Invest Smarter with One Super App      Operat...
extracted from URL 182:   Building the Future: A Comprehensive Overview of Singapore...
extracted from URL 183: Singapore's real estate rally predicted to halt, says Morgan...
extracted from URL 184: My Analysis of Frasers Logistics & Commercial Trust - The Si...
extracted from URL 185:  Over 50 advanced manufacturing innovations to launch at Ind...
extracted from URL 186: Asia-Pacific real estate market remains resilient in 2024 de...
extracted from URL 187: How digitalisation is fueling Singapore’s economy: I&C secto...
extracted from URL 188: Ubi Industrial Property Review - YouTubeAboutPressCopyrightC...
extracted from URL 189: Singapore Switchgear Market | Grow at 10.65% CAGR till 2030T...
extracted from URL 190: Singapore and Changi Airport eyeing a busier – and greener –...
                                                   URL  Quarter  \
180  https://www

In [7]:
import pandas as pd

file_path = 'all_extracted_texts.xlsx' 
sheet_name = '2020 - 2024'
df = pd.read_excel(file_path, sheet_name=sheet_name)
df_cleaned = df.drop_duplicates(subset='URLs', keep='first')

df_cleaned.to_excel("cleaned_extracted_texts.xlsx", index=False)

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

df = pd.read_excel('web_scraping_urls.xlsx', sheet_name='master')

def clean_text(text):
    return re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)

def extract_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return clean_text(soup.get_text())
    except requests.RequestException as e:
        return str(e)

batch_size = 10
output_filename = 'all_extracted_texts.xlsx'

with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    total_urls = len(df['URLs'])
    for start in range(900, total_urls, batch_size):
        end = start + batch_size
        print(f"processing batch from URL {start+1} to {min(end, total_urls)}")
        batch_data = {'URL': df['URLs'][start:end], 'Quarter': df['Quarter'][start:end], 'Extracted Text': []}

        for index, url in enumerate(df['URLs'][start:end], start=start+1):
            text = extract_text(url)
            batch_data['Extracted Text'].append(text)
            print(f"extracted from URL {index}: {text[:60]}...")
            time.sleep(2)

        batch_df = pd.DataFrame(batch_data)
        print(batch_df.head())  # Print head of the DataFrame to check it
        batch_df.to_excel(writer, sheet_name='master', index=False, header=not writer.sheets, startrow=writer.sheets['master'].max_row if 'master' in writer.sheets else 0)

        print(f"batch from URL {start+1} to {min(end, total_urls)} saved.")


processing batch from URL 901 to 910
extracted from URL 901: Resource Sustainability Act: Singapore's road to zero wasteS...
extracted from URL 902: Sembcorp acquires Veolia's public cleaning business for $28 ...
extracted from URL 903: YASC Building Extension & Renovation: Safety Blessing Ceremo...
extracted from URL 904: 403 Client Error: Forbidden for url: https://www.propwise.sg...
extracted from URL 905: Economic headwinds may moderate industrial rents in 2020 - S...
extracted from URL 906: 403 Client Error: Forbidden for url: https://www.colliers.co...
extracted from URL 907: LogisticsSkip to main                    content            ...
extracted from URL 908: Latest Commercial Property News | Office Spaces Singapore Ho...
extracted from URL 909: 404 Client Error: Not Found for url: https://sg.finance.yaho...
extracted from URL 910: %PDF-1.7%����1 0 obj<>/Metadata 3135 0 R/ViewerPreferences 3...
                                                   URL  Quarter  \
900  https://www

KeyboardInterrupt: 

## Date Extraction

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

# Load the Excel file
df = pd.read_excel('web_scraping_urls.xlsx', sheet_name='2020 - 2024')

# List to hold the dates
dates = []

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to extract date from HTML with a timeout
def get_date_from_html(url):
    try:
        # Set a timeout for the request to 30 seconds
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        date_pattern = r'\b\d{4}-\d{2}-\d{2}\b'
        date_string = re.search(date_pattern, soup.get_text())
        if date_string:
            return datetime.strptime(date_string.group(), '%Y-%m-%d').date()
        else:
            return None
    except requests.exceptions.Timeout:
        print(f"Timeout occurred for {url}")
        return None
    except Exception as e:
        print(f"Failed to process {url}: {e}")
        return None

# Loop through each URL, extract the date, and append to the list
for index, url in enumerate(df['URLs'], start=1):
    print(f"Processing URL {index} of {len(df['URLs'])}: {url}")
    date = get_date_from_html(url)
    dates.append(date)

# Add dates to the DataFrame
df['Extracted Date'] = dates

# Save the DataFrame back to the same Excel file
df.to_excel('all_extracted_dates.xlsx', index=False)

Processing URL 1 of 903: https://www.colliers.com/en-sg/news/2024-02-14-q4-singapore-industrial-market-update-press#:~:text=Report%20highlights&text=The%20price%20index%20rose%20by,growth%20and%20weakness%20in%20exports.
Processing URL 2 of 903: https://www.businesstimes.com.sg/property/bt-property-week-2024/singapore-industrial-strong-growth-prospects-no-easy-ride
Processing URL 3 of 903: https://www.colliers.com/en-sg/research/2023-q4-singapore-industrial-report-colliers
Processing URL 4 of 903: https://www.jll.com.sg/en/newsroom/moderation-in-industrial-rent-and-price-growth-in-4q23
Processing URL 5 of 903: https://www.edgeprop.sg/property-news/industrial-rents-prices-rose-13th-straight-quarter-4q2023
Processing URL 6 of 903: https://sbr.com.sg/commercial-property/news/industrial-rents-grow-0-3-in-2024-report
Processing URL 7 of 903: https://www.savills.com.sg/insight-and-opinion/savills-news/215105-0/industrial-rental-growth-rate-between-0--to-3--in-2024
Processing URL 8 of 903: ht

In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

df = pd.read_excel('web_scraping_urls.xlsx', sheet_name='2020 - 2024')

dates = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
date_patterns = [
    r'\b\d{4}-\d{2}-\d{2}\b',  # YYYY-MM-DD
    r'\b\d{2}/\d{2}/\d{4}\b',  # DD/MM/YYYY
    r'\b\w+ \d{1,2}, \d{4}\b'  # Month DD, YYYY
]

def get_date_from_html(url):
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for pattern in date_patterns:
            date_string = re.search(pattern, soup.get_text())
            if date_string:
                # Attempt to parse the date, handling different formats
                try:
                    return datetime.strptime(date_string.group(), '%Y-%m-%d').date()
                except ValueError:
                    try:
                        return datetime.strptime(date_string.group(), '%d/%m/%Y').date()
                    except ValueError:
                        return datetime.strptime(date_string.group(), '%B %d, %Y').date()
        return None
    except requests.exceptions.Timeout:
        print(f"timeout occurred for {url}")
        return None
    except Exception as e:
        print(f"failed to process {url}: {e}")
        return None
    
for index, url in enumerate(df['URLs'], start=1):
    print(f"processing URL {index} of {len(df['URLs'])}: {url}")
    date = get_date_from_html(url)
    print(date)
    dates.append(date if date is not None else "No Date Found")

df['Extracted Date'] = dates

df.to_excel('all_extracted_dates.xlsx', index=False)

processing URL 1 of 903: https://www.colliers.com/en-sg/news/2024-02-14-q4-singapore-industrial-market-update-press#:~:text=Report%20highlights&text=The%20price%20index%20rose%20by,growth%20and%20weakness%20in%20exports.
failed to process https://www.colliers.com/en-sg/news/2024-02-14-q4-singapore-industrial-market-update-press#:~:text=Report%20highlights&text=The%20price%20index%20rose%20by,growth%20and%20weakness%20in%20exports.: time data 'Feb 13, 2024' does not match format '%B %d, %Y'
None
processing URL 2 of 903: https://www.businesstimes.com.sg/property/bt-property-week-2024/singapore-industrial-strong-growth-prospects-no-easy-ride
failed to process https://www.businesstimes.com.sg/property/bt-property-week-2024/singapore-industrial-strong-growth-prospects-no-easy-ride: time data 'Feb 27, 2024' does not match format '%B %d, %Y'
None
processing URL 3 of 903: https://www.colliers.com/en-sg/research/2023-q4-singapore-industrial-report-colliers
failed to process https://www.colliers

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

def get_date_from_html(url):
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Look for <time> tags first
        time_tag = soup.find('time')
        if time_tag and time_tag.get('datetime'):
            return datetime.strptime(time_tag['datetime'], '%Y-%m-%d').date()
        elif time_tag:
            # Try to parse the text inside the time tag
            return parse_date_from_string(time_tag.text)

        # If no <time> tag, look for specific classes or ids
        date_span = soup.find('span', class_='date-class')  # Adjust class name as needed
        if date_span:
            return parse_date_from_string(date_span.text)

        # Fallback to regex if specific tags not found
        date_string = search_dates_with_regex(soup.get_text())
        if date_string:
            return date_string
        
        return None
    except requests.exceptions.Timeout:
        print(f"Timeout occurred for {url}")
        return None
    except Exception as e:
        print(f"Failed to process {url}: {e}")
        return None

def parse_date_from_string(date_str):
    for fmt in ('%Y-%m-%d', '%d/%m/%Y', '%B %d, %Y'):
        try:
            return datetime.strptime(date_str.strip(), fmt).date()
        except ValueError:
            continue
    return None

def search_dates_with_regex(text):
    date_patterns = [
        r'\b\d{4}-\d{2}-\d{2}\b',
        r'\b\d{2}/\d{2}/\d{4}\b',
        r'\b\w+ \d{1,2}, \d{4}\b'
    ]
    for pattern in date_patterns:
        date_match = re.search(pattern, text)
        if date_match:
            return parse_date_from_string(date_match.group())
    return None

# Place your existing functions here (get_date_from_html, parse_date_from_string, search_dates_with_regex)

# Adjust headers if necessary
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def read_excel_file(file_path):
    return pd.read_excel(file_path)

def write_excel_file(data, file_path):
    data.to_excel(file_path, index=False)

def add_dates_to_dataframe(df, url_column):
    # Adding a new column for the extracted dates
    df['Extracted Date'] = df[url_column].apply(lambda url: get_date_from_html(url) if pd.notna(url) else None)
    return df

# Adjust the file paths and column name as needed
input_file_path = 'path_to_your_input_excel_file.xlsx'
output_file_path = 'path_to_your_output_excel_file.xlsx'
url_column_name = 'URL'  # Adjust this to the actual URL column name in your Excel file

# Process the Excel file
df = read_excel_file(input_file_path)
df_with_dates = add_dates_to_dataframe(df, url_column_name)
write_excel_file(df_with_dates, output_file_path)